# **Youtube Videos Transcription with Faster Whisper**


[![notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)](https://colab.research.google.com/github/lewangdev/whisper-youtube/blob/main/faster_whisper_youtube.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/lewangdev/faster_whisper_youtube)](https://github.com/lewangdev/faster_whisper_youtube)


[faster-whisper](https://github.com/guillaumekln/faster-whisper) is a reimplementation of OpenAI's Whisper model using CTranslate2, which is a fast inference engine for Transformer models.

This implementation is up to 4 times faster than openai/whisper for the same accuracy while using less memory. The efficiency can be further improved with 8-bit quantization on both CPU and GPU.

Whisper is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification.

This Notebook will guide you through the transcription of a Youtube video using Faster Whisper. You'll be able to explore most inference parameters or use the Notebook as-is to store the transcript and video audio in your Google Drive.

In [ ]:
#@markdown # **Check GPU type** 🕵️

#@markdown The type of GPU you get assigned in your Colab session defined the speed at which the video will be transcribed.
#@markdown The higher the number of floating point operations per second (FLOPS), the faster the transcription.
#@markdown But even the least powerful GPU available in Colab is able to run any Whisper model.
#@markdown Make sure you've selected `GPU` as hardware accelerator for the Notebook (Runtime &rarr; Change runtime type &rarr; Hardware accelerator).

#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@markdown ---
#@markdown **Factory reset your Notebook's runtime if you want to get assigned a new GPU.**

!nvidia-smi -L

!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-7e38a915-a2fd-d236-3367-8aea40007d5e)
Wed Nov 22 09:20:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |   2091MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

In [ ]:
#@markdown # **Install libraries** 🏗️
#@markdown This cell will take a little while to download several libraries, including Faster Whisper.

#@markdown ---

! pip install faster-whisper
! pip install yt-dlp

import sys
import warnings
from faster_whisper import WhisperModel
from pathlib import Path
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

Using device: cuda:0


In [ ]:
#@markdown # **Optional:** Save data in Google Drive 💾
#@markdown Enter a Google Drive path and run this cell if you want to store the results inside Google Drive.

# Uncomment to copy generated images to drive, faster than downloading directly from colab in my experience.
from google.colab import drive
drive_mount_path = Path("/") / "content" / "drive"
drive.mount(str(drive_mount_path))
drive_mount_path /= "My Drive"
#@markdown ---
drive_path = "\u7F85\u6D5A/done_txt" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change your Google Drive path.**

drive_whisper_path = drive_mount_path / Path(drive_path.lstrip("/"))
drive_whisper_path.mkdir(parents=True, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@markdown # **Model selection** 🧠

#@markdown As of the first public release, there are 4 pre-trained options to play with:

#@markdown |  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
#@markdown |:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
#@markdown |  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~0.8 GB     |      ~32x      |
#@markdown |  base  |    74 M    |     `base.en`      |       `base`       |     ~1.0 GB     |      ~16x      |
#@markdown | small  |   244 M    |     `small.en`     |      `small`       |     ~1.4 GB     |      ~6x       |
#@markdown | medium |   769 M    |    `medium.en`     |      `medium`      |     ~2.7 GB     |      ~2x       |
#@markdown | large-v1  |   1550 M   |        N/A         |      `large-v1`       |    ~4.3 GB     |       1x       |
#@markdown | large-v2  |   1550 M   |        N/A         |      `large-v2`       |    ~4.3 GB     |       1x       |

#@markdown ---
model_size = 'large-v2' #@param ['tiny', 'tiny.en', 'base', 'base.en', 'small', 'small.en', 'medium', 'medium.en', 'large-v1', 'large-v2']
device_type = "cuda" #@param {type:"string"} ['cuda', 'cpu']
compute_type = "float16" #@param {type:"string"} ['float16', 'int8_float16', 'int8']
#@markdown ---
#@markdown **Run this cell again if you change the model.**

model = WhisperModel(model_size, device=device_type, compute_type=compute_type)


In [ ]:
#@markdown # **Video selection** 📺

#@markdown Enter the URL of the Youtube video you want to transcribe, wether you want to save the audio file in your Google Drive, and run the cell.

Type = "Google Drive" #@param ['Youtube video or playlist', 'Google Drive']
#@markdown ---
#@markdown #### **Youtube video or playlist**
URL = "" #@param {type:"string"}
# store_audio = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **Google Drive video, audio (mp4, wav), or folder containing video and/or audio files**
video_path = "\u7F85\u6D5A/\u5F85\u8F49\u9010\u5B57\u7A3F" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change the video.**

video_path_local_list = []

if Type == "Youtube video or playlist":

    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',
        # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
        'postprocessors': [{  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([URL])
        list_video_info = [ydl.extract_info(URL, download=False)]

    for video_info in list_video_info:
        video_path_local_list.append(Path(f"{video_info['id']}.wav"))

elif Type == "Google Drive":
    # video_path_drive = drive_mount_path / Path(video_path.lstrip("/"))
    video_path = drive_mount_path / Path(video_path.lstrip("/"))
    if video_path.is_dir():
        for video_path_drive in video_path.glob("**/*"):
            if video_path_drive.is_file():
                display(Markdown(f"**{str(video_path_drive)} selected for transcription.**"))
            elif video_path_drive.is_dir():
                display(Markdown(f"**Subfolders not supported.**"))
            else:
                display(Markdown(f"**{str(video_path_drive)} does not exist, skipping.**"))
            video_path_local = Path(".").resolve() / (video_path_drive.name)
            shutil.copy(video_path_drive, video_path_local)
            video_path_local_list.append(video_path_local)
    elif video_path.is_file():
        video_path_local = Path(".").resolve() / (video_path.name)
        shutil.copy(video_path, video_path_local)
        video_path_local_list.append(video_path_local)
        display(Markdown(f"**{str(video_path)} selected for transcription.**"))
    else:
        display(Markdown(f"**{str(video_path)} does not exist.**"))

else:
    raise(TypeError("Please select supported input type."))

for video_path_local in video_path_local_list:
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])


In [ ]:
def seconds_to_time_format(s):
    # Convert seconds to hours, minutes, seconds, and milliseconds
    hours = s // 3600
    s %= 3600
    minutes = s // 60
    s %= 60
    seconds = s // 1
    milliseconds = round((s % 1) * 1000)

    # Return the formatted string
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"

#@markdown # **Run the model** 🚀

#@markdown Run this cell to execute the transcription of the video. This can take a while and very based on the length of the video and the number of parameters of the model selected above.

#@markdown ## **Parameters** ⚙️

#@markdown ### **Behavior control**
#@markdown #### Language
language = "auto" #@param ["auto", "en", "zh", "ja", "fr", "de"] {allow-input: true}
#@markdown #### initial prompt
initial_prompt = "Please do not translate, only transcription be allowed.  Here are some English words you may need: Cindy. And Chinese words: \u7206\u7834" #@param {type:"string"}
#@markdown ---
#@markdown #### Word-level timestamps
word_level_timestamps = False #@param {type:"boolean"}
#@markdown ---
#@markdown #### VAD filter
vad_filter = True #@param {type:"boolean"}
vad_filter_min_silence_duration_ms = 50 #@param {type:"integer"}
#@markdown ---
#@markdown #### Output(Default is srt, txt if `text_only` be checked )
text_only = True #@param {type:"boolean"}


# 遍歷 video_path_local_list，為每個文件進行轉錄
for video_path_local in video_path_local_list:
    # 轉換 MP4 文件到 WAV 格式（如果需要）
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])
    # 轉換 WMA 文件到 WAV 格式（如果需要）
    elif video_path_local.suffix == ".wma":
        video_path_local = video_path_local.with_suffix(".wav")
        subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".wma")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])

    # 執行轉錄
    segments, info = model.transcribe(str(video_path_local), beam_size=5,
                                      language=None if language == "auto" else language,
                                      initial_prompt=initial_prompt,
                                      word_timestamps=word_level_timestamps,
                                      vad_filter=vad_filter,
                                      vad_parameters=dict(min_silence_duration_ms=vad_filter_min_silence_duration_ms))

    display(Markdown(f"Detected language '{info.language}' with probability {info.language_probability}"))

    # 生成逐字稿文件
    ext_name = '.txt' if text_only else ".srt"
    transcript_file_name = video_path_local.stem + ext_name
    sentence_idx = 1
    with open(transcript_file_name, 'w') as f:
        for segment in segments:
            if word_level_timestamps:
                for word in segment.words:
                    ts_start = seconds_to_time_format(word.start)
                    ts_end = seconds_to_time_format(word.end)
                    print(f"[{ts_start} --> {ts_end}] {word.word}")
                    if not text_only:
                        f.write(f"{sentence_idx}\n")
                        f.write(f"{ts_start} --> {ts_end}\n")
                        f.write(f"{word.word}\n\n")
                    else:
                        f.write(f"{word.word} ")
                    sentence_idx += 1
            else:
                ts_start = seconds_to_time_format(segment.start)
                ts_end = seconds_to_time_format(segment.end)
                print(f"[{ts_start} --> {ts_end}] {segment.text}")
                if not text_only:
                    f.write(f"{sentence_idx}\n")
                    f.write(f"{ts_start} --> {ts_end}\n")
                    f.write(f"{segment.text.strip()}\n\n")
                else:
                    f.write(f"{segment.text.strip()}\n")
                sentence_idx += 1
        if text_only:
            f.write("\n")  # 確保在文本模式下換行


    try:
        shutil.copy(video_path_local.parent / transcript_file_name,
                    drive_whisper_path / transcript_file_name
        )
        display(Markdown(f"**Transcript file created: {drive_whisper_path / transcript_file_name}**"))
    except:
        display(Markdown(f"**Transcript file created: {video_path_local.parent / transcript_file_name}**"))


Detected language 'en' with probability 0.98486328125

[00:00:01,000 --> 00:00:04,350]  Thank you for waiting. The meeting will begin shortly.
[00:00:04,350 --> 00:00:08,500]  Good afternoon, everyone. I'm Casey.
[00:00:08,500 --> 00:00:15,100]  Welcome to join us for First Financial Holdings 3rd Quarter 2020 Webcast Investor Conference.
[00:00:15,100 --> 00:00:23,020]  Before we proceed today's presentation, I'd like to disclose the following information.
[00:00:23,020 --> 00:00:31,340]  Starting from December 2015, in order to improve cooperation, First Financial has set up...
[00:00:31,340 --> 00:00:41,000]  For more information, please refer to our website.
[00:00:41,000 --> 00:00:46,420]  Okay, let's start with our performance presentation.
[00:00:46,420 --> 00:00:55,810]  Today's material can be downloaded from our website today after today's conference.
[00:00:55,810 --> 00:01:06,720]  And after today's conference, we will talk about...
[00:01:06,720 --> 00:01:18,190]  For participants...
[00:01:18,190 --> 00:01:19,190]  Window.
[00

**Transcript file created: /content/drive/My Drive/羅浚/done_txt/2892第一金_2020Q3法人說明會(英)_20201130.txt**

Detected language 'en' with probability 0.95703125

[00:00:01,000 --> 00:00:04,000]  Thank you for waiting. We will begin our meeting today.
[00:00:04,000 --> 00:00:07,000]  Good afternoon, ladies and gentlemen. Hi, everyone.
[00:00:07,000 --> 00:00:23,150]  Welcome to join us for First Financial Holdings First Half 2019 Webcast Investor Conference.
[00:00:23,150 --> 00:00:31,470]  Then we will proceed, which will be hosted by…
[00:00:31,470 --> 00:00:39,880]  And for investors, you can raise your questions by typing at the bottom of the…
[00:00:39,880 --> 00:00:45,830]  And today's presentation, you can download it from our website or from the…
[00:00:45,830 --> 00:00:51,300]  And also we will provide one-year replay.
[00:00:51,300 --> 00:01:00,260]  After today, Mr. Keith Kerr to begin today's presentation. Keith.
[00:01:00,260 --> 00:01:04,540]  Thank you, Casey. Please turn to slide.
[00:01:04,540 --> 00:01:23,130]  First Financial Holdings, $9 billion NTD.
[00:01:23,130 --> 00:01:27,130]  Among the subsidiaries, bank unit grew by 5

...

[00:00:00,500 --> 00:00:05,500]  Welcome everyone to Firestone 2012 Q2 earnings conference call.
[00:00:05,500 --> 00:00:08,500]  All lines have been placed on mute to prevent the background noise.
[00:00:08,500 --> 00:00:12,500]  After the presentation, there will be a question and answer session.
[00:00:12,500 --> 00:00:16,500]  Please follow the instructions given at that time if you would like to ask a question.
[00:00:16,500 --> 00:00:22,500]  For your information, a webcast replay will be available within an hour after the conference is finished.
[00:00:22,500 --> 00:00:29,500]  Please visit www.firestone.com.tw under the investor relations section.
[00:00:29,500 --> 00:00:33,500]  Now, I would like to introduce Mr. Gary Lai, the IR officer.
[00:00:33,500 --> 00:00:35,500]  Gary, you may begin.
[00:00:35,500 --> 00:00:42,660]  Thank you everybody to attend Firestone Q2 2012 conference call again.
[00:00:42,660 --> 00:00:49,660]  Today, you know, again, our President Yvonne Lee an

**Transcript file created: /content/drive/My Drive/羅浚/done_txt/4904遠傳_2012Q2法人說明會(英)_20120725.txt**

Detected language 'en' with probability 0.98876953125

[00:00:01,000 --> 00:00:05,000]  Welcome everyone to the Forest Zone 2012 Q100 conference call.
[00:00:05,000 --> 00:00:08,000]  All lines have been placed on mute to prevent background noise.
[00:00:08,000 --> 00:00:12,000]  After the presentation, there will be a question and answer section.
[00:00:12,000 --> 00:00:16,000]  Please follow the instructions given at that time if you would like to ask a question.
[00:00:16,000 --> 00:00:22,100]  For your information, a webcast replay will be available within an hour after the conference is finished.
[00:00:23,100 --> 00:00:34,100]  Please visit www.fareastone.com.tw under the Investor Relations section.
[00:00:34,100 --> 00:00:38,450]  I would like to introduce Mr. Gary Lai, our IR officer.
[00:00:38,450 --> 00:00:40,450]  Gary, you may begin.
[00:00:40,450 --> 00:00:47,450]  Thank you everybody to attend our first quarter 2012 investor conference call.
[00:00:47,450 --> 00:00:52,450]  Our president Yvonne and CFO TY are all here for us 

**Transcript file created: /content/drive/My Drive/羅浚/done_txt/4904遠傳_2012Q1法人說明會(英)_20120420.txt**